# Table of Contents
### 1. Importing Libraries & Data
### 2. Addressing PII Data
### 3. Create a regional segmentation of the data
### 4. Create an exclusion flag for low-activity customers (less than 5 orders) and exclude them from the data
### 5.1 Creating age groups
### 5.2 Creating income groups
### 5.3 Creating groups based on number of dependants
### 6. Exporting Data


# 1. Importing Libraries & Data

In [2]:
# Importing Libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
# Creating Path
path = r'/Users/mattwawner/Documents/8-2022-Instacart Basket Analysis'

In [4]:
# Importing 'orders_products_customers_merged' as ords_prods_custs_merge

ords_prods_custs_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers_merged.pkl'))

# 2. Addressing PII Data

In [5]:
# Checking for columns with PII data

ords_prods_custs_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,...,med_days_since_prior_order,regularity_flag,last_name,Gender,State,Age,date_joined,number_of_dependants,fam_status,income
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,8.0,Frequent customer,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,8.0,Frequent customer,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,1,5,13,...,8.0,Frequent customer,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,...,8.0,Frequent customer,Cox,Male,Minnesota,81,8/1/2019,1,married,49620
4,1000,Apricots,18,10,12.9,505689,138,9,6,12,...,8.0,Frequent customer,Cox,Male,Minnesota,81,8/1/2019,1,married,49620


In [6]:
# Creating a dataframe with no PII data by dropping 'last_name' column from 'ords_custs_prods_merge'

ords_prods_custs_merge_no_pii = ords_prods_custs_merge.drop(columns = ['last_name'])

# 3. Create a regional segmentation of the data

In [7]:
# Creating regions for data to be grouped into 

Northeast = ['Maine','New Hampshire', 'Vermont', 'Massachusetts', 
             'Rhode Island', 'Connecticut', 'New York',
             'Pennsylvania', 'New Jersey']
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 
          'Ohio', 'North Dakota', 'South Dakota', 'Nebraska',
           'Kansas', 'Minnesota', 'Iowa', 'Missouri']
South = ['Delaware', 'Maryland', 'District of Columbia', 
        'Virginia', 'West Virginia', 'North Carolina', 
        'South Carolina', 'Georgia', 'Florida', 'Kentucky', 
        'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma',
        'Texas', 'Arkansas', 'Louisiana']
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado',
       'Arizona', 'New Mexico', 'Alaska', 'Washington', 
       'Oregon', 'California', 'Hawaii']

In [8]:
# Labeling states in the Northeast as 'Northeast' in new 'Region' column

ords_prods_custs_merge_no_pii.loc[ords_prods_custs_merge_no_pii['State'].isin(Northeast), 'Region'] = 'Northeast'

In [9]:
# Labeling states in the Midwest as 'Midwest' in new 'Region' column

ords_prods_custs_merge_no_pii.loc[ords_prods_custs_merge_no_pii['State'].isin(Midwest), 'Region'] = 'Midwest'

In [10]:
# Labeling states in the South as 'South' in new 'Region' column

ords_prods_custs_merge_no_pii.loc[ords_prods_custs_merge_no_pii['State'].isin(South), 'Region'] = 'South'

In [11]:
# Labeling states in the West as 'West' in new 'Region' column

ords_prods_custs_merge_no_pii.loc[ords_prods_custs_merge_no_pii['State'].isin(West), 'Region'] = 'West'

In [14]:
# Checking the number of orders from each region

ords_prods_custs_merge_no_pii['Region'].value_counts(dropna = False)

South        10792300
West          8293217
Midwest       7597585
Northeast     5722939
Name: Region, dtype: int64

In [11]:
# Checking the new 'Region' column

ords_prods_custs_merge_no_pii.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,...,med_days_since_prior_order,regularity_flag,Gender,State,Age,date_joined,number_of_dependants,fam_status,income,Region
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,1,5,13,...,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,...,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest
4,1000,Apricots,18,10,12.9,505689,138,9,6,12,...,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest


In [12]:
# Isolating 'Region' & 'spending_flag' columns to examine relationship

crosstab = pd.crosstab(ords_prods_custs_merge_no_pii['spending_flag'], ords_prods_custs_merge_no_pii['Region'], dropna = False)

In [13]:
# Copying 'Region' & 'spending_flag' data to clipboard to be posted in Excel

crosstab.to_clipboard()

In [14]:
# Checking the distribution of high & low spenders in the different regions

crosstab

Region,Midwest,Northeast,South,West
spending_flag,,,,
High spender,155979,108226,209693,160354
Low spender,7441606,5614713,10582607,8132863


In [15]:
crosstab2 = pd.crosstab(ords_prods_custs_merge_no_pii['prices'], ords_prods_custs_merge_no_pii['Region'], dropna = False)

In [16]:
crosstab2

Region,Midwest,Northeast,South,West
prices,,,,
1.0,27384,20681,38022,29056
1.1,57198,43728,80284,61572
1.2,60178,44838,85759,65483
1.3,99420,74462,142621,109188
1.4,91861,69875,130542,99931
...,...,...,...,...
24.8,84,62,147,117
24.9,1511,886,1848,1536
25.0,3193,2541,4440,3598


#### Spending habits do not seem to significantly vary between regions. The percentage of 'high spenders' in each region is as follows: Midwest = 2.05%, South = 1.94%, West = 1.93%, Northeast = 1.89%. 

In [17]:
ords_prods_custs_merge_no_pii.dtypes

product_id                       int64
product_name                    object
aisle_id                         int64
department_id                    int64
prices                         float64
order_id                         int64
user_id                         object
order_number                     int64
order_day_of_week                int64
order_hour_of_day                int64
days_since_prior_order         float64
add_to_cart_order                int64
reordered                        int64
_merge                        category
price_range_loc                 object
busiest_day                     object
busiest_days_of_week            object
busiest_hours_of_day            object
max_order                        int64
loyalty_flag                    object
avg_prod_price                 float64
spending_flag                   object
med_days_since_prior_order     float64
regularity_flag                 object
Gender                          object
State                    

# 4. Create an exclusion flag for low-activity customers (less than 5 orders) and exclude them from the data

In [18]:
# Creating an 'total_orders' column to derive a low_activity flag from

ords_prods_custs_merge_no_pii['total_orders'] = ords_prods_custs_merge_no_pii.groupby(['user_id'])['order_number'].transform(np.max)

In [19]:
# Checking if new columns 'total_orders' was created

ords_prods_custs_merge_no_pii.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,...,regularity_flag,Gender,State,Age,date_joined,number_of_dependants,fam_status,income,Region,total_orders
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,32
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,32
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,1,5,13,...,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,32
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,...,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,32
4,1000,Apricots,18,10,12.9,505689,138,9,6,12,...,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,32


In [20]:
# Flagging users with total_orders < 5 as 'Low-activity customer'

ords_prods_custs_merge_no_pii.loc[ords_prods_custs_merge_no_pii['total_orders'] < 5,
        'activity_flag'] = 'Low-activity customer'

In [21]:
# Flagging users with total_orders >= 5 as 'Regular-activity customer'

ords_prods_custs_merge_no_pii.loc[ords_prods_custs_merge_no_pii['total_orders'] >= 5,
        'activity_flag'] = 'Regular-activity customer'

In [22]:
ords_prods_custs_merge_no_pii.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,...,Gender,State,Age,date_joined,number_of_dependants,fam_status,income,Region,total_orders,activity_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,32,Regular-activity customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,32,Regular-activity customer
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,1,5,13,...,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,32,Regular-activity customer
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,...,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,32,Regular-activity customer
4,1000,Apricots,18,10,12.9,505689,138,9,6,12,...,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,32,Regular-activity customer


In [23]:
# Creating a subset of low_activity_customers

low_activity_customers = ords_prods_custs_merge_no_pii[ords_prods_custs_merge_no_pii['activity_flag']=='Low-activity customer']

In [24]:
# Exporting data

low_activity_customers.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'low_activity_customers.pkl'))

In [25]:
# Excluding 'Low-activity customers' from dataset and naming it 'opc_clean'

opc_clean = ords_prods_custs_merge_no_pii[ords_prods_custs_merge_no_pii['activity_flag']=='Regular-activity customer']

In [26]:
# Checking to see if rows were removed

ords_prods_custs_merge_no_pii.shape

(32406041, 34)

In [27]:
low_activity_customers.shape

(1440355, 34)

In [48]:
opc_clean.shape

(30965686, 37)

In [47]:
# Prices > $100 were replaced with 'NaN' in 4.9 but data was never exported, so step is being completed before aggregation occurs 

opc_clean.loc[opc_clean['prices'] > 100, 'prices'] = np.nan

# 5.1 Creating age groups: 
##### - Young adults(<30 yrs.)
##### - Middle-age adults (30-64 yrs.)
##### - Seniors (>65 yrs.)

In [30]:
opc_clean.tail()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,...,Gender,State,Age,date_joined,number_of_dependants,fam_status,income,Region,total_orders,activity_flag
32406025,49235,Organic Half & Half,53,16,1.8,3310701,27382,5,3,9,...,Female,Vermont,68,11/25/2019,3,married,37867,Northeast,9,Regular-activity customer
32406026,49235,Organic Half & Half,53,16,1.8,527883,27382,6,1,12,...,Female,Vermont,68,11/25/2019,3,married,37867,Northeast,9,Regular-activity customer
32406027,49235,Organic Half & Half,53,16,1.8,685496,27382,7,4,8,...,Female,Vermont,68,11/25/2019,3,married,37867,Northeast,9,Regular-activity customer
32406028,49235,Organic Half & Half,53,16,1.8,1224680,27382,8,1,7,...,Female,Vermont,68,11/25/2019,3,married,37867,Northeast,9,Regular-activity customer
32406029,49235,Organic Half & Half,53,16,1.8,2223687,27382,9,2,12,...,Female,Vermont,68,11/25/2019,3,married,37867,Northeast,9,Regular-activity customer


In [31]:
# Creating a young adult group

opc_clean.loc[opc_clean['Age'] < 30,
        'age_group'] = 'Young adult'

/var/folders/rj/0rb3kwqs2vd37lmgjltm01qc0000gn/T/ipykernel_38801/3373152295.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opc_clean.loc[opc_clean['Age'] < 30,


In [32]:
# Creating a middle-age adult group

opc_clean.loc[(opc_clean['Age'] >= 30) & (opc_clean['Age'] < 65),
         'age_group'] = 'Middle-age adult'

In [33]:
# Creating a senior age group

opc_clean.loc[opc_clean['Age'] >= 65,
        'age_group'] = 'Senior'

In [34]:
# Checking if procedure was successful

opc_clean.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,...,State,Age,date_joined,number_of_dependants,fam_status,income,Region,total_orders,activity_flag,age_group
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Minnesota,81,8/1/2019,1,married,49620,Midwest,32,Regular-activity customer,Senior
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Minnesota,81,8/1/2019,1,married,49620,Midwest,32,Regular-activity customer,Senior
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,1,5,13,...,Minnesota,81,8/1/2019,1,married,49620,Midwest,32,Regular-activity customer,Senior
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,...,Minnesota,81,8/1/2019,1,married,49620,Midwest,32,Regular-activity customer,Senior
4,1000,Apricots,18,10,12.9,505689,138,9,6,12,...,Minnesota,81,8/1/2019,1,married,49620,Midwest,32,Regular-activity customer,Senior


In [35]:
# Checking the distribution of age groups

opc_clean['age_group'].value_counts()

Middle-age adult    16952029
Senior               8195806
Young adult          5817851
Name: age_group, dtype: int64

#### It can be seen that the majority of customers are betweent the ages of 30-64. The age group of 65 years and older is the 2nd largest, with young adults under the age of 30 being the smallest.

In [52]:
# Aggregating the max, mean, and min on a customer-profile level for expenditure

opc_clean.groupby('age_group').agg({'prices':['mean', 'min', 'max']})

prices           
                      mean  min   max
age_group                            
Middle-age adult  7.790474  1.0  25.0
Senior            7.796694  1.0  25.0
Young adult       7.782162  1.0  25.0

#### Middle-age adults and seniors spend almost the same amounts, while young adults spend slightly less.

In [53]:
# Aggregating the max, mean, and min on a customer-profile level for order frequency

opc_clean.groupby('age_group').agg({'days_since_prior_order':['mean', 'min', 'max']})

days_since_prior_order           
                                   mean  min   max
age_group                                         
Middle-age adult              10.612932  0.0  30.0
Senior                        10.587780  0.0  30.0
Young adult                   10.610895  0.0  30.0

#### Middle-age adults and young adults order at approximately the same frequency, while seniors order slightly less often.

# 5.2 Creating income groups:
##### - Low-income : 0-60,000
##### - Middle-income : 60,000-120,000
##### - High-income : 120,000+

In [37]:
# Creating a low-income group

opc_clean.loc[opc_clean['income'] < 60000,
'income_group'] = 'Low-income'

/var/folders/rj/0rb3kwqs2vd37lmgjltm01qc0000gn/T/ipykernel_38801/687058247.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opc_clean.loc[opc_clean['income'] < 60000,


In [38]:
# Creating a middle-income group

opc_clean.loc[(opc_clean['income'] >= 60000) & (opc_clean['income'] < 120000),
         'income_group'] = 'Middle-income'

In [39]:
# Creating a high-income group

opc_clean.loc[opc_clean['income'] > 120000,
        'income_group'] = 'High-income'

In [40]:
# Checking the distribution of the new 'income_group' column

opc_clean['income_group'].value_counts()

Middle-income    15982610
High-income       9179831
Low-income        5802974
Name: income_group, dtype: int64

#### It can be seen that the majority of customers are in the middle-income group.

In [50]:
# Aggregating the max, mean, and min on a customer-profile level for expenditure

opc_clean.groupby('income_group').agg({'prices':['mean', 'min', 'max']})

prices           
                   mean  min   max
income_group                      
High-income    7.952086  1.0  25.0
Low-income     7.083489  1.0  25.0
Middle-income  7.954546  1.0  25.0

#### On average, the high-income and middle-income groups purchase around the same priced items, while the low-income group purchases slightly less expensive products on average, by approximately $0.87.

In [51]:
# Aggregating the max, mean, and min on a customer-profile level for order frequency

opc_clean.groupby('income_group').agg({'days_since_prior_order':['mean', 'min', 'max']})

days_since_prior_order           
                                mean  min   max
income_group                                   
High-income                10.559700  0.0  30.0
Low-income                 10.822304  0.0  30.0
Middle-income              10.553867  0.0  30.0

#### It seems that the high & middle-income groups order slightly more often than the low-income group.

# 5.3 Creating groups based on number of dependants

In [41]:
# Creating a No dependants group

opc_clean.loc[opc_clean['number_of_dependants'] == 0,
        'dependant_flag'] = 'No dependants'

/var/folders/rj/0rb3kwqs2vd37lmgjltm01qc0000gn/T/ipykernel_38801/1487723075.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opc_clean.loc[opc_clean['number_of_dependants'] == 0,


In [42]:
# Creating a Has depedants group

opc_clean.loc[opc_clean['number_of_dependants'] > 0,
        'dependant_flag'] = 'Has dependant(s)'

In [43]:
# Checking the distribution of the new 'dependant_flag' column

opc_clean['dependant_flag'].value_counts()

Has dependant(s)    23225670
No dependants        7740016
Name: dependant_flag, dtype: int64

#### It can be seen that the majority of customers have dependants

In [49]:
# Aggregating the max, mean, and min on a customer-profile level for prices

opc_clean.groupby('dependant_flag').agg({'prices':['mean', 'min', 'max']})

prices           
                      mean  min   max
dependant_flag                       
Has dependant(s)  7.790904  1.0  25.0
No dependants     7.789521  1.0  25.0

#### On average, those who have dependants spend slightly more than those who do not.

In [54]:
# Aggregating the max, mean, and min on a customer-profile level for prices

opc_clean.groupby('dependant_flag').agg({'days_since_prior_order':['mean', 'min', 'max']})

days_since_prior_order           
                                   mean  min   max
dependant_flag                                    
Has dependant(s)              10.611609  0.0  30.0
No dependants                 10.588737  0.0  30.0

#### User with no dependants order slightly more frequently than those who have dependant(s)

# 6. Exporting Data

In [45]:
# Exporting data
opc_clean.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'opc_clean.pkl'))